In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bitcoin-tweets/Bitcoin_tweets_dataset_2.csv
/kaggle/input/bitcoin-tweets/Bitcoin_tweets.csv


In [2]:
!pip install --upgrade scikit-learn numpy pandas joblib optuna xgboost lightgbm matplotlib seaborn tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.0.3
    Uninstalling xgboost-2.0.3:
      Successfully uninstalled xgboost-2.0.3
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Succ

In [3]:
import csv
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer

# Download necessary NLTK data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('vader_lexicon', quiet=True)

# Initialize NLTK tools
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
sia = SentimentIntensityAnalyzer()

def preprocess_text(text):
    if not isinstance(text, str):
        if pd.isna(text):
            return ''
        text = str(text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stopwords and apply stemming
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    
    return ' '.join(tokens)

def read_and_process_csv(file_path, chunk_size=10000):
    total_tweets = 0
    unique_tweets = set()
    processed_chunks = []
    current_chunk = []

    with open(file_path, 'r', encoding='utf-8', errors='replace') as file:
        # Read the header
        header = next(csv.reader([file.readline()]))
        text_column_index = header.index('text') if 'text' in header else None

        if text_column_index is None:
            print("Error: 'text' column not found in the CSV file.")
            return None, 0, set()

        csv_reader = csv.reader((line.replace('\0', '') for line in file))
        for row in tqdm(csv_reader, desc="Preprocessing tweets"):
            try:
                if len(row) > text_column_index:
                    tweet_text = row[text_column_index]
                    processed_text = preprocess_text(tweet_text)
                    current_chunk.append([tweet_text, processed_text])
                    unique_tweets.add(tweet_text)
                    
                    if len(current_chunk) >= chunk_size:
                        df_chunk = pd.DataFrame(current_chunk, columns=['text', 'processed_text'])
                        processed_chunks.append(df_chunk)
                        total_tweets += len(current_chunk)
                        current_chunk = []
                else:
                    print(f"Skipping row due to insufficient columns: {row}")
            except Exception as e:
                print(f"Error processing row: {e}")
                continue

    if current_chunk:
        df_chunk = pd.DataFrame(current_chunk, columns=['text', 'processed_text'])
        processed_chunks.append(df_chunk)
        total_tweets += len(current_chunk)

    df = pd.concat(processed_chunks, ignore_index=True)
    return df, total_tweets, unique_tweets

# File path to your raw tweet dataset in Kaggle
file_path = '/kaggle/input/bitcoin-tweets/Bitcoin_tweets_dataset_2.csv'
df_raw, total_tweets, unique_tweets = read_and_process_csv(file_path)

if df_raw is not None:
    # Save preprocessed data to Kaggle working directory
    df_raw.to_csv('/kaggle/working/preprocessed_bitcoin_tweets.csv', index=False)
    print("\nPreprocessed data saved to '/kaggle/working/preprocessed_bitcoin_tweets.csv'")
    print(f"Total tweets processed: {total_tweets}")

    # Sentiment labeling using VADER
    def analyze_sentiment(text):
        sentiment_scores = sia.polarity_scores(text)
        compound_score = sentiment_scores['compound']
        if compound_score >= 0.05:
            return 'positive'
        elif compound_score <= -0.05:
            return 'negative'
        else:
            return 'neutral'

    # Apply sentiment analysis (using original text)
    df_raw['sentiment'] = df_raw['text'].apply(analyze_sentiment)
    
    # Save sentiment-labeled data
    df_raw.to_csv('/kaggle/working/sentiment_labeled_bitcoin_tweets.csv', index=False)
    print("Sentiment analysis completed and saved to '/kaggle/working/sentiment_labeled_bitcoin_tweets.csv'")
    
    # Display some statistics
    print("\nSentiment Distribution:")
    print(df_raw['sentiment'].value_counts())
else:
    print("Failed to process the CSV file.")

Preprocessing tweets: 800it [00:00, 2738.31it/s]

Skipping row due to insufficient columns: ['👉https://t.co/dwh8jR9uho']
Skipping row due to insufficient columns: ['$ALPH 3.16%']
Skipping row due to insufficient columns: ['$BDP 3.05%']
Skipping row due to insufficient columns: ['$BLOCK 0.77%']
Skipping row due to insufficient columns: ['$ALI 0.41%']
Skipping row due to insufficient columns: ['$CFX 0.4%']
Skipping row due to insufficient columns: [' #ALPH #BDP #BLOCK #ALI #CFX #crypto #cryptonews #cryptoAlerts #blockchain  #cryptotrading #cryptoprices #altcoins #bitcoin', "['cryptocurrency', 'price', 'GateIo', 'ALPH', 'BDP', 'BLOCK', 'ALI', 'CFX', 'crypto', 'cryptonews', 'cryptoAlerts', 'blockchain', 'cryptotrading', 'cryptoprices', 'altcoins', 'bitcoin']", 'Crypto Frog', 'False']


Preprocessing tweets: 1684it [00:00, 2837.07it/s]

Skipping row due to insufficient columns: ['👉https://t.co/XsdBPI7qf6']
Skipping row due to insufficient columns: ['$ORB 3.74%']
Skipping row due to insufficient columns: ['$KAR 0.95%']
Skipping row due to insufficient columns: ['$CHAT 0.65%']
Skipping row due to insufficient columns: ['$LAT 0.48%']
Skipping row due to insufficient columns: ['$CMT 0.4%']
Skipping row due to insufficient columns: [' #ORB #KAR #CHAT #LAT #CMT #crypto #cryptonews #cryptoAlerts #blockchain  #cryptotrading #cryptoprices #altcoins #bitcoin', "['cryptocurrency', 'price', 'OKX', 'ORB', 'KAR', 'CHAT', 'LAT', 'CMT', 'crypto', 'cryptonews', 'cryptoAlerts', 'blockchain', 'cryptotrading', 'cryptoprices', 'altcoins', 'bitcoin']", 'Crypto Frog', 'False']
Skipping row due to insufficient columns: ['👉https://t.co/dwh8jR9uho']
Skipping row due to insufficient columns: ['$ALPH 4.49%']
Skipping row due to insufficient columns: ['$LQTY 0.94%']
Skipping row due to insufficient columns: ['$VELO 0.72%']
Skipping row due to ins

Preprocessing tweets: 3730it [00:01, 2914.04it/s]

...

Skipping row due to insufficient columns: []
Skipping row due to insufficient columns: ['Via https://t.co/p6iec02QvX']
Skipping row due to insufficient columns: []
Skipping row due to insufficient columns: ['#btc    #4chan  #cryptocurrency https://t.co/5EPrDHSjig', "['btc', '4chan', 'cryptocurrency']", 'topstonks', 'False']


Preprocessing tweets: 174274it [00:59, 2871.11it/s]

Skipping row due to insufficient columns: []
Skipping row due to insufficient columns: ['Via https://t.co/p6iec02QvX']
Skipping row due to insufficient columns: []
Skipping row due to insufficient columns: ['#btc    #4chan  #cryptocurrency https://t.co/162sQTXjHj', "['btc', '4chan', 'cryptocurrency']", 'topstonks', 'False']
Skipping row due to insufficient columns: []
Skipping row due to insufficient columns: ['Via https://t.co/p6iec02QvX']
Skipping row due to insufficient columns: []
Skipping row due to insufficient columns: ['#btc    #4chan  #cryptocurrency https://t.co/KeMQ6uj6bw', "['btc', '4chan', 'cryptocurrency']", 'topstonks', 'False']
Skipping row due to insufficient columns: []
Skipping row due to insufficient columns: ['Via https://t.co/p6iec02QvX']
Skipping row due to insufficient columns: []
Skipping row due to insufficient columns: ['#btc    #4chan  #altcoin https://t.co/38603Icnac', "['btc', '4chan', 'altcoin']", 'topstonks', 'False']


Preprocessing tweets: 174438it [00:59, 2938.59it/s]



Preprocessed data saved to '/kaggle/working/preprocessed_bitcoin_tweets.csv'
Total tweets processed: 169820
Sentiment analysis completed and saved to '/kaggle/working/sentiment_labeled_bitcoin_tweets.csv'

Sentiment Distribution:
sentiment
neutral     75181
positive    68995
negative    25644
Name: count, dtype: int64


In [4]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import save_npz
import numpy as np
import pandas as pd

# Load the sentiment-labeled data
df = pd.read_csv('/kaggle/working/sentiment_labeled_bitcoin_tweets.csv')
df_clean = df.dropna(subset=['processed_text'])
print(f"Removed {len(df) - len(df_clean)} rows with missing processed text.")

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_clean['processed_text'])

# Save TF-IDF matrix and vectorizer to Kaggle working directory
save_npz('/kaggle/working/tfidf_matrix.npz', tfidf_matrix)
joblib.dump(tfidf_vectorizer, '/kaggle/working/tfidf_vectorizer.joblib')

# Save feature names
feature_names = tfidf_vectorizer.get_feature_names_out()
np.save('/kaggle/working/tfidf_feature_names.npy', feature_names)

print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")
print(f"Number of features: {len(feature_names)}")

Removed 1 rows with missing processed text.
TF-IDF matrix shape: (169819, 5000)
Number of features: 5000


In [5]:
import numpy as np
import pandas as pd
from scipy.sparse import load_npz
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from tqdm import tqdm
import optuna
import time

# Load TF-IDF matrix and sentiment-labeled data
tfidf_matrix = load_npz('/kaggle/working/tfidf_matrix.npz')
df = pd.read_csv('/kaggle/working/sentiment_labeled_bitcoin_tweets.csv')
df_clean = df.dropna(subset=['processed_text'])
sentiments = df_clean['sentiment']

# Encode string labels into integers
le = LabelEncoder()
encoded_labels = le.fit_transform(sentiments)

# Hyperparameter Tuning for XGBoost using Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('xgb_n_estimators', 100, 300),
        'learning_rate': trial.suggest_float('xgb_learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('xgb_max_depth', 3, 12)
    }
    
    xgb_trial = XGBClassifier(
        random_state=42, 
        eval_metric='mlogloss',
        tree_method='hist',    
        device='cuda',         
        **params
    )
    
    estimators_trial = [
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)),
        ('xgb', xgb_trial),
        ('lgbm', LGBMClassifier(
            boosting_type='gbdt',
            objective='multiclass',
            n_estimators=200,
            learning_rate=0.1,
            device='gpu'
        ))
    ]
    
    stacking_trial = StackingClassifier(
        estimators=estimators_trial,
        final_estimator=LogisticRegression(),
        cv=5,
        n_jobs=1
    )
    
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    for train_idx, test_idx in cv.split(tfidf_matrix, encoded_labels):
        X_train = tfidf_matrix[train_idx].copy()
        X_test = tfidf_matrix[test_idx].copy()
        y_train, y_test = encoded_labels[train_idx], encoded_labels[test_idx]
        stacking_trial.fit(X_train, y_train)
        scores.append(stacking_trial.score(X_test, y_test))
    return np.mean(scores)

print("Starting hyperparameter tuning for XGBoost using Optuna...")
study = optuna.create_study(direction="maximize")
start_time = time.time()
study.optimize(objective, n_trials=5)
print(f"Hyperparameter tuning completed in {time.time() - start_time:.2f} seconds.")
print("Best XGBoost parameters found:", study.best_trial.params)

# Build Final Stacked Ensemble with Tuned XGBoost
best_params = study.best_trial.params
xgb_final = XGBClassifier(
    random_state=42, 
    eval_metric='mlogloss',
    tree_method='hist',
    device='cuda',
    n_estimators=best_params['xgb_n_estimators'],
    learning_rate=best_params['xgb_learning_rate'],
    max_depth=best_params['xgb_max_depth']
)

rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
lgbm = LGBMClassifier(
    boosting_type='gbdt',
    objective='multiclass',
    n_estimators=200,
    learning_rate=0.1,
    device='gpu'
)

estimators = [
    ('rf', rf),
    ('xgb', xgb_final),
    ('lgbm', lgbm)
]

stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    cv=5,
    n_jobs=1
)

# Cross-validation of the Final Stacked Ensemble
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

print("\nPerforming cross-validation with the Final Stacked Ensemble...")
for train_index, test_index in tqdm(cv.split(tfidf_matrix, encoded_labels), total=cv.get_n_splits(), desc="CV Folds"):
    X_train = tfidf_matrix[train_index].copy()
    X_test  = tfidf_matrix[test_index].copy()
    y_train, y_test = encoded_labels[train_index], encoded_labels[test_index]
    
    stacking_clf.fit(X_train, y_train)
    score = stacking_clf.score(X_test, y_test)
    cv_scores.append(score)

print("\nFinal Stacked Ensemble - Cross-validation Accuracy:")
print(f"Mean accuracy: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores)*2:.4f})")

# Train the Final Stacked Ensemble on the Full Dataset and Evaluate
print("\nTraining the Final Stacked Ensemble on the full dataset...")
stacking_clf.fit(tfidf_matrix.copy(), encoded_labels)

predictions = stacking_clf.predict(tfidf_matrix.copy())
print("\nClassification Report (Final Stacked Ensemble):")
print(classification_report(encoded_labels, predictions, target_names=le.classes_))

cm = confusion_matrix(encoded_labels, predictions)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix - Final Stacked Ensemble')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.savefig('/kaggle/working/final_stacked_confusion_matrix.png')
plt.close()

# Save the Final Model and Label Encoder
joblib.dump(stacking_clf, '/kaggle/working/final_stacked_model.joblib')
joblib.dump(le, '/kaggle/working/label_encoder.joblib')

print("\nModel training complete. The final stacked ensemble model and label encoder have been saved.")

[I 2025-04-20 13:51:34,953] A new study created in memory with name: no-name-80774f83-36c9-48d6-a5b2-dfb2f6ff54c2


Starting hyperparameter tuning for XGBoost using Optuna...
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 219747
[LightGBM] [Info] Number of data points in the train set: 113212, number of used features: 4693
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 2 dense feature groups (0.43 MB) transferred to GPU in 0.000778 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score -1.890418
[LightGBM] [Info] Start training from score -0.814842
[LightGBM] [Info] Start training from score -0.900708


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:729: UserWarning: [13:58:32] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


...

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 250186
[LightGBM] [Info] Number of data points in the train set: 135855, number of used features: 4878
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 2 dense feature groups (0.52 MB) transferred to GPU in 0.000901 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score -1.890432
[LightGBM] [Info] Start training from score -0.814846
[LightGBM] [Info] Start training from score -0.900698


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 248990
[LightGBM] [Info] Number of data points in the train set: 135856, number of used features: 4854
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 2 dense feature groups (0.52 MB) transferred to GPU in 0.001271 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score -1.890390
[LightGBM] [Info] Start training from score -0.814854
[LightGBM] [Info] Start training from score -0.900705


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



Classification Report (Final Stacked Ensemble):
              precision    recall  f1-score   support

    negative       0.94      0.91      0.93     25644
     neutral       0.96      0.97      0.97     75180
    positive       0.96      0.96      0.96     68995

    accuracy                           0.96    169819
   macro avg       0.95      0.95      0.95    169819
weighted avg       0.96      0.96      0.96    169819


Model training complete. The final stacked ensemble model and label encoder have been saved.
